In [1]:
!pip install transformers datasets sentencepiece torch accelerate peft bitsandbytes

In [2]:
import torch
import pandas as pd
import numpy as np
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

 Разбиваем датасет на части

In [3]:
import psutil
import torch

def get_optimal_chunk_size(min_chunk_size=10000, max_chunk_size=50000):
    """
    Определяет оптимальный размер чанка на основе доступной оперативной памяти (RAM) и GPU-памяти (VRAM).
    
    :param min_chunk_size: Минимальный размер чанка (по умолчанию 10 000 записей)
    :param max_chunk_size: Максимальный размер чанка (по умолчанию 50 000 записей)
    :return: Оптимальный размер чанка
    """
    # Получаем доступную оперативную память (RAM) в ГБ
    total_ram = psutil.virtual_memory().available / (1024 ** 3)  # В ГБ
    
    # Определяем доступную видеопамять (VRAM) в ГБ, если есть GPU
    if torch.cuda.is_available():
        total_vram = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)  # В ГБ
    else:
        total_vram = 0  # Если GPU нет, считаем, что VRAM = 0

    # Рассчитываем примерный размер чанка
    estimated_chunk_size = int((total_ram + total_vram) * 2500)  # Чем больше памяти, тем больше чанки

    # Ограничиваем размер чанка диапазоном (min_chunk_size - max_chunk_size)
    optimal_chunk_size = max(min(estimated_chunk_size, max_chunk_size), min_chunk_size)

    print(f"💾 Доступная RAM: {total_ram:.2f} ГБ | 🎮 Доступная VRAM: {total_vram:.2f} ГБ")
    print(f"🔹 Оптимальный размер чанка: {optimal_chunk_size} записей")
    
    return optimal_chunk_size

# Загружаем и предобрабатываем данные
df = pd.read_csv("output_final.csv", sep=";").dropna()

# Переименовываем колонки
if "text_wich_errors" in df.columns:
    df.rename(columns={"text_wich_errors": "text_with_errors"}, inplace=True)

# 📌 Определяем динамический размер чанка перед разбиением датасета
chunk_size = get_optimal_chunk_size()

# Разбиваем датасет на части (чанки)
for i, start in enumerate(range(0, len(df), chunk_size)):
    df.iloc[start:start + chunk_size].to_csv(f"chunk_{i + 1}.csv", sep=";", index=False)

print(f"✅ Датасет разбит на {i + 1} частей с размером чанка {chunk_size}")

💾 Доступная RAM: 52.52 ГБ | 🎮 Доступная VRAM: 11.66 ГБ
🔹 Оптимальный размер чанка: 50000 записей
✅ Датасет разбит на 5 частей с размером чанка 50000


Настройка модели (ruT5-large)

In [4]:
MODEL_NAME = "ai-forever/ruT5-large"

# 8-битное квантование
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Токенизатор
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, legacy=False)

# Загружаем модель
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto"
)

# Отключаем кеширование
model.config.use_cache = False


Настройка LoRA

In [5]:
# Конфигурация LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)
model = get_peft_model(model, lora_config)

# Фиксируем параметры модели, чтобы они обучались
for param in model.parameters():
    if param.dtype in [torch.float16, torch.float32]:
        param.requires_grad = True



Функции предобработки

In [6]:
# Функции предобработки данных
def preprocess_data(example):
    return {
        "input_text": "Исправь текст: " + example["text_with_errors"],
        "target_text": example["corrected_text"]
    }

def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    labels = tokenizer(
        examples["target_text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



Настройка обучения

In [7]:
# Настройка аргументов для обучения
checkpoint_dir = "./ruT5-corrector-checkpoints"

training_args = TrainingArguments(
    output_dir=checkpoint_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    weight_decay=0.01,
    bf16=True,
    logging_dir="./logs",
    logging_steps=500,
    optim="adamw_bnb_8bit",
    label_names=["labels"],
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


# Функция для вычисления метрик
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return {"accuracy": np.mean([p == l for p, l in zip(decoded_preds, decoded_labels)])}


/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Обучение

In [8]:
# Обучение модели на чанках
chunks = sorted([f for f in os.listdir() if f.startswith("chunk_") and f.endswith(".csv")])

for i, chunk in enumerate(chunks):
    print(f"\n🔹 Обучение на части {i + 1}/{len(chunks)}: {chunk}")

    # Загрузка данных из чанка
    df_chunk = pd.read_csv(chunk, sep=";").dropna()
    dataset = Dataset.from_pandas(df_chunk)
    
    # Разделяем данные на обучающую и тестовую выборки
    dataset_split = dataset.train_test_split(test_size=0.1, seed=42)
    train_dataset = dataset_split["train"]
    test_dataset = dataset_split["test"]

    # Применяем предобработку данных
    train_dataset = train_dataset.map(preprocess_data)
    test_dataset = test_dataset.map(preprocess_data)

    # Токенизация данных
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)

    # Создаём тренера
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    # Обучаем модель
    trainer.train()

    # Сохраняем модель
    trainer.save_model(f"{checkpoint_dir}/checkpoint-{i + 1}")




🔹 Обучение на части 1/5: chunk_1.csv


Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/opt/conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.02 GiB. GPU 0 has a total capacity of 11.66 GiB of which 2.02 GiB is free. Process 4495 has 27.63 MiB memory in use. Process 3613596 has 4.88 GiB memory in use. Process 3709684 has 4.64 GiB memory in use. Of the allocated memory 3.84 GiB is allocated by PyTorch, and 897.12 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)